In [ ]:
pip install opencv-python==3.4.18.65

In [1]:
import cv2
import numpy as np

In [2]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# detector_params = cv2.SimpleBlobDetector_Params()
# detector_params.filterByArea = True
# detector_params.maxArea = 1500
# detector = cv2.SimpleBlobDetector_create(detector_params)

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Change thresholds
params.minThreshold = 0.2;
params.maxThreshold = 1;
 




# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints


def nothing(x):
    pass


def main():
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('image')
    cv2.createTrackbar('threshold', 'image', 0, 255, nothing)
    while True:
        _, frame = cap.read()
        threshold = r = cv2.getTrackbarPos('threshold', 'image')
        
        # Create a detector with the parameters
        params.filterByCircularity = True
        params.minCircularity = threshold
        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
          detector = cv2.SimpleBlobDetector(params)
        else : 
          detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(frame)
        
        eye = cv2.drawKeypoints(frame, keypoints, frame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv2.imshow('image', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

error: OpenCV(3.4.18) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
